In [ ]:
import os
import shutil
from PIL import Image
import json
import numpy as np
import random

random.seed(2024)

In [ ]:
# 폴더 경로 설정
train_folder = '/data/ephemeral/home/sroie/train'  
test_folder = '/data/ephemeral/home/sroie/test'

# 이미지 파일 개수 확인
train_img_list = os.listdir(f'{train_folder}/img')
test_img_list = os.listdir(f'{test_folder}/img')
print('train 이미지 개수 :', len(train_img_list))
print('test 이미지 개수 :', len(test_img_list))

# annotation 파일 개수 확인
train_box_list = os.listdir(f'{train_folder}/box')
test_box_list = os.listdir(f'{test_folder}/box')
print('train box 개수 :', len(train_box_list))
print('test box 개수 :', len(test_box_list))

In [ ]:
"""
set_sroie_from_original

캐글에서 다운로드 받은 원본 데이터에 새로운 number를 붙여서 sroie_receipt 데이터를 만든다.

input
    json_name : annotation을 저장할 json 파일 이름
    only_json : 이미지는 저장하지 않고 json 파일만 추가하는 경우
    n_sample : random sampling해서 annotation에 추가할 데이터 개수(전체 추가 시 none)
"""
def set_sroie_from_original(sroie_dir='/data/ephemeral/home/sroie', save_dir='/data/ephemeral/home/code/data', json_name='train.json', 
                            only_json=False, n_sample=None, worker="worker", timestamp="2024-11-01"):
    
    img_dict = {'train': None, 
                'test': None}
    
    # 이미지 불러오기 및 총 sample 개수 세기
    total_sample = 0
    for split in img_dict.keys():
        img_dict[split] = os.listdir(f'{sroie_dir}/{split}/img')
        total_sample += len(img_dict[split])

    # 난수 생성
    if n_sample is not None:
        random_list = random.sample(range(total_sample), n_sample)

    # 이미지를 복사할 대상 폴더
    img_save_folder = f'{save_dir}/sroie_receipt/img/train'
    ufo_save_folder = f'{save_dir}/sroie_receipt/ufo'

    # 대상 폴더가 존재하지 않으면 생성
    os.makedirs(img_save_folder, exist_ok=True)
    os.makedirs(ufo_save_folder, exist_ok=True)

    num = 0 # new numbering
    train_json = {"images": dict()}

    for split in img_dict.keys():
        img_list = img_dict[split]

        for i in range(len(img_list)):
                
            name = img_list[i][:-4] # X00016469612
            img_source_path = f'{sroie_dir}/{split}/img/{name}.jpg'

            # image copy
            save_name = f"extractor.sroie.{num:04}.jpg" # "extractor.sroie.0000.jpg"
            if only_json == False:
                img_save_path = f"{img_save_folder}/{save_name}"
                shutil.copy(img_source_path, img_save_path)

            # random list에 속하는 순번에만 json에 추가
            if n_sample is not None and num not in random_list:
                num += 1
                continue
            num += 1

            # image size
            image = Image.open(img_source_path)
            width, height = image.size

            # points
            box_source_path = f'{sroie_dir}/{split}/box/{name}.txt'
            
            with open(box_source_path, 'r', encoding='utf-8', errors='replace') as txt:
                lines = txt.readlines()

            words_dict = dict()
            for j in range(len(lines)):
                line = lines[j].strip() # 줄마다 \n 제거
                if line == "":
                    continue
                line = line.split(",")
                transcription = ",".join(line[8:])
                points = []
                for point in line[:8]:
                    points.append(float(point))
                points = np.reshape(np.array(points), (4, 2)).tolist()
                words_dict[f"{j+1:04}"] = {"transcription": transcription,
                                           "points": points}

            # 이미지 별 정보 저장
            train_json["images"][name] = {"paragraphs": {},
                                          "words": words_dict,
                                          "chars": {},
                                          "img_w": width,
                                          "img_h": height,
                                          "num_patches": None,
                                          "tags": ['sroie'],
                                          "relations": {},
                                          "annotation_log": {
                                              "worker": worker,
                                              "timestamp": timestamp,
                                              "tool_version": "",
                                              "source": None
                                          },
                                          "license_tag": {
                                              "usability": True,
                                              "public": True,
                                              "commercial": True,
                                              "type": "CC BY 4.0",
                                              "holder": "SROIE"
                                          }
                                          }

    with open(f"{ufo_save_folder}/{json_name}", 'w', encoding='utf-8') as js:
        json.dump(train_json, js, indent=4)

    print(f'{len(train_json["images"])} samples saved')

In [ ]:
set_sroie_from_original(sroie_dir='.', save_dir='.', json_name='train_random.json',
                        only_json=False, n_sample=400, worker="Kim Yoonseo", timestamp="2024-11-02")

In [ ]:
"""
set_sroie_from_clean

set_sroie_from_original로 만든 새로운 데이터셋에서 data cleaning을 위해 이미지 몇 개를 삭제했을 경우, 
이미지를 기준으로 다시 sroie_receipt 데이터를 만든다. 
"""
def set_sroie_from_clean(sroie_dir='/data/ephemeral/home/code/data/sroie_receipt', save_dir='/data/ephemeral/home/code/data', json_name='train.json', 
                         only_json=True, n_sample=400, worker="worker", timestamp="2024-11-04"):
    
    img_dict = {'train': None}
    
    # 이미지 불러오기 및 총 sample 개수 세기
    total_sample = 0
    for split in img_dict.keys():
        img_dict[split] = os.listdir(f'{sroie_dir}/img/{split}')
        total_sample += len(img_dict[split])
    
    with open(f"{sroie_dir}/ufo/train.json", 'r', encoding='utf8') as ufo:
        ufo_dict = json.load(ufo)

    # 난수 생성
    if n_sample is not None:
        random_list = random.sample(range(total_sample), n_sample)

    # 이미지를 복사할 대상 폴더
    img_save_folder = f'{save_dir}/sroie_receipt/img/train'
    ufo_save_folder = f'{save_dir}/sroie_receipt/ufo'

    # 대상 폴더가 존재하지 않으면 생성
    if only_json == False:
        os.makedirs(img_save_folder, exist_ok=True)
    os.makedirs(ufo_save_folder, exist_ok=True)

    num = 0 # not use in numbering, only in random
    train_json = {"images": dict()}

    for split in img_dict.keys():
        img_list = img_dict[split]

        for i in range(len(img_list)):
                
            name = img_list[i] # extractor.sroie.0000.jpg
            img_source_path = f'{sroie_dir}/img/{split}/{name}'

            # image copy
            if only_json == False:
                img_save_path = f"{img_save_folder}/{name}"
                shutil.copy(img_source_path, img_save_path)

            # random list에 속하는 순번에만 json에 추가
            if n_sample is not None and num not in random_list:
                num += 1
                continue
            num += 1

            # image size
            image = Image.open(img_source_path)

            # points
            train_json["images"][name] = ufo_dict["images"][name]

            train_json["images"][name]["annotation_log"]["worker"] = worker
            train_json["images"][name]["annotation_log"]["timestamp"] = timestamp

    with open(f"{ufo_save_folder}/{json_name}", 'w', encoding='utf-8') as js:
        json.dump(train_json, js, indent=4)

    print(f'{len(train_json["images"])} samples saved')

In [ ]:
set_sroie_from_clean(sroie_dir='./sroie_receipt_clean_img', save_dir='.', json_name='train_random.json',
                     only_json=False, n_sample=None, worker="Kim Yoonseo", timestamp="2024-11-04")